In [2]:
import psycopg2

In [17]:
host="157.230.240.246"
database="learn_psycopg2"
user="iykra_trainee"
password="passwordiykra"

In [46]:
#conn.close()
conn = psycopg2.connect(host=host,database=database, user=user, password=password)
cur = conn.cursor()

In [25]:
cur.execute("""
SELECT table_schema,table_name
FROM information_schema.tables
ORDER BY table_schema,table_name;""")
cur.fetchall()

[('information_schema', 'administrable_role_authorizations'),
 ('information_schema', 'applicable_roles'),
 ('information_schema', 'attributes'),
 ('information_schema', 'character_sets'),
 ('information_schema', 'check_constraint_routine_usage'),
 ('information_schema', 'check_constraints'),
 ('information_schema', 'collation_character_set_applicability'),
 ('information_schema', 'collations'),
 ('information_schema', 'column_domain_usage'),
 ('information_schema', 'column_options'),
 ('information_schema', 'column_privileges'),
 ('information_schema', 'columns'),
 ('information_schema', 'column_udt_usage'),
 ('information_schema', 'constraint_column_usage'),
 ('information_schema', 'constraint_table_usage'),
 ('information_schema', 'data_type_privileges'),
 ('information_schema', 'domain_constraints'),
 ('information_schema', 'domains'),
 ('information_schema', 'domain_udt_usage'),
 ('information_schema', 'element_types'),
 ('information_schema', 'enabled_roles'),
 ('information_sche

In [11]:
def create_tables(conn,cur):
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE vendors_coba_1 (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(%s) NOT NULL
        )
        """ % ('255'))
    for command in commands:
        cur.execute(command)
        conn.commit()
        cur.close()

In [14]:
def create_tables(conn,cur):
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE vendors_1 (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts_1 (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings_1 (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts_1 (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)
    for command in commands:
        cur.execute(command)
    cur.close()
    conn.commit()

In [15]:
create_tables(conn,cur)

In [22]:
def insert_vendor(vendor_name,conn,cur):
    """ insert a new vendor into the vendors table """
    sql = """INSERT INTO vendors(vendor_name)
             VALUES(%s) RETURNING vendor_id;"""
    vendor_id = None
    try:
        # execute the INSERT statement
        cur.execute(sql, (vendor_name,))
        # get the generated id back
        vendor_id = cur.fetchone()[0]
        # commit the changes to the database
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
 
    return vendor_id

In [40]:
def insert_vendor_list(vendor_list,conn,cur):
    """ insert multiple vendors into the vendors table  """
    sql = "INSERT INTO vendors(vendor_name) VALUES(%s)"
    try:
        # execute the INSERT statement
        cur.executemany(sql,vendor_list)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [48]:
# insert one vendor
#insert_vendor("Iykra Corp.",conn,cur)
# insert multiple vendors
insert_vendor_list([
    ('Garuda Terbang Inc.',),
    ('Dodol Co Ltd.',)
],conn,cur)

In [99]:
#sql = 'INSERT INTO vendors(vendor_name) VALUES('Garuda Terbang 1 Inc.','Dodol 1 Co Ltd.')' % ()
s = ['Garuda Terbang 20 Inc.','Dodol 20 Co Ltd.']
sql = "INSERT INTO vendors(%s) VALUES %s" % (",".join("('"+x+"')" for x in s))
cur.execute(sql)
conn.commit()
#("Garuda Terbang 5 Inc."),("Garuda Terbang 6 Inc.")



In [112]:
syntax = "select * from vendors"
cur.execute(syntax)
a = cur.fetchall()
a_id = [x[0] for x in a]
a_vendornm = [x[1] for x in a]

In [116]:
import pandas as pd

In [124]:
gepeng4 = pd.DataFrame({'id' : a_id, 'vendor_name' : a_vendornm})
gepeng4.drop_duplicates(['vendor_name'],inplace = False)

,id,vendor_name
0,16,AKM Semiconductor Inc.
1,17,Asahi Glass Co Ltd.
2,18,Daikin Industries Ltd.
3,19,Dynacast International Inc.
4,20,Foster Electric Co. Ltd.
5,21,Murata Manufacturing Co. Ltd.
6,15,3M Corp
19,34,Iykra Corp.
20,35,Garuda Indonesia Inc.
21,36,Fanta Co Ltd.


In [125]:
gepeng4

,id,vendor_name
0,16,AKM Semiconductor Inc.
1,17,Asahi Glass Co Ltd.
2,18,Daikin Industries Ltd.
3,19,Dynacast International Inc.
4,20,Foster Electric Co. Ltd.
5,21,Murata Manufacturing Co. Ltd.
6,15,3M Corp
7,22,AKM Semiconductor Inc.
8,23,Asahi Glass Co Ltd.
9,24,Daikin Industries Ltd.


In [77]:
#s = ['Garuda Terbang Inc.','Dodol Co Ltd.']
#" ".join(x for x in tuple(s))
#tuple(s
s = ['Garuda Terbang Inc.','Dodol Co Ltd.']


[('Garuda Terbang Inc.', 'Dodol Co Ltd.')]

In [62]:
column_value = ['Garuda Terbang Inc.','Dodol Co Ltd.']
tuple(column_value) 

TypeError: 'tuple' object is not callable

In [34]:
def update_vendor(vendor_id, vendor_name,conn,cur):
    """ update vendor name based on the vendor id """
    sql = """ UPDATE vendors
                SET vendor_name = %s
                WHERE vendor_id = %s"""
    updated_rows = 0
    try:
        cur.execute(sql, (vendor_name, vendor_id))
        # get the number of updated rows
        updated_rows = cur.rowcount
        # Commit the changes to the database
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    return updated_rows

In [36]:
def get_vendors(con,cur):
    """ query data from the vendors table """
    try:
        cur.execute("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")
        print("The number of parts: ", cur.rowcount)
        row = cur.fetchone()
 
        while row is not None:
            print(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [39]:
cur.execute("select * from public.vendors")
cur.fetchall()

[(16, 'AKM Semiconductor Inc.'),
 (17, 'Asahi Glass Co Ltd.'),
 (18, 'Daikin Industries Ltd.'),
 (19, 'Dynacast International Inc.'),
 (20, 'Foster Electric Co. Ltd.'),
 (21, 'Murata Manufacturing Co. Ltd.'),
 (15, '3M Corp')]

In [40]:
def delete_part(part_id):
    """ delete part by part id """
    conn = None
    rows_deleted = 0
    try:
        # execute the UPDATE  statement
        cur.execute("DELETE FROM parts WHERE part_id = %s", (part_id,))
        # get the number of updated rows
        rows_deleted = cur.rowcount
        # Commit the changes to the database
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
 
    return rows_deleted